In [67]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

In [11]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [12]:
db, coll = connectCollection('companies','companies')

In [138]:
# companies = list(coll.find()) # 18801

# Query en compass: {$or:[{"offices.latitude":{$ne:null}},{"offices.longitude":{$ne:null}}]}
# En python es None en lugar de null.

companies = list(coll.find({'$or':[{"offices.latitude":{'$ne':None}},{"offices.longitude":{'$ne':None}}]})) # 14082

In [139]:
len(companies) # 14082

14082

In [140]:
companies[0]

{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
 'name': 'AdventNet',
 'permalink': 'abc3',
 'crunchbase_url': 'http://www.crunchbase.com/company/adventnet',
 'homepage_url': 'http://adventnet.com',
 'blog_url': '',
 'blog_feed_url': '',
 'twitter_username': 'manageengine',
 'category_code': 'enterprise',
 'number_of_employees': 600,
 'founded_year': 1996,
 'deadpooled_year': 2,
 'tag_list': '',
 'alias_list': 'Zoho ManageEngine ',
 'email_address': 'pr@adventnet.com',
 'phone_number': '925-924-9500',
 'description': 'Server Management Software',
 'created_at': datetime.datetime(2007, 5, 25, 19, 24, 22),
 'updated_at': 'Wed Oct 31 18:26:09 UTC 2012',
 'overview': '<p>AdventNet is now <a href="/company/zoho-manageengine" title="Zoho ManageEngine" rel="nofollow">Zoho ManageEngine</a>.</p>\n\n<p>Founded in 1996, AdventNet has served a diverse range of enterprise IT, networking and telecom customers.</p>\n\n<p>AdventNet supplies server and network management software.</p>',
 'image': {'avail

In [141]:
def set_key(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = value
    elif type(dictionary[key]) == list:
        dictionary[key].append(value)
    else:
        dictionary[key] = [dictionary[key], value]

In [143]:
companies_dict = {}
for i in range(len(companies)):
    set_key(companies_dict, '_id', companies[i]['_id'])
    set_key(companies_dict, 'name', companies[i]['name'])
    set_key(companies_dict, 'founded_year', companies[i]['founded_year'])
    set_key(companies_dict, 'category_code', companies[i]['category_code'])
    set_key(companies_dict, 'total_money_raised', companies[i]['total_money_raised'])
    set_key(companies_dict, 'deadpooled_year', companies[i]['deadpooled_year'])

In [ ]:
# print(len(companies_dict['id'])) # 14082
# print(len(companies_dict['name'])) # 14082
# print(len(companies_dict['founded_year'])) # 14082
# print(len(companies_dict['category_code'])) # 14082
# print(len(companies_dict['total_money_raised'])) # 14082
# print(len(companies_dict['deadpooled_year'])) # 14082
# print(companies_dict.keys())

In [144]:
companies_df = pd.DataFrame.from_dict(companies_dict)

In [145]:
companies_df.head(10)

,_id,name,founded_year,category_code,total_money_raised,deadpooled_year
0,52cdef7c4bab8bd675297d8b,AdventNet,1996.0,enterprise,$0,2.0
1,52cdef7c4bab8bd675297d8c,Zoho,2005.0,software,$0,3.0
2,52cdef7c4bab8bd675297d8a,Wetpaint,2005.0,web,$39.8M,1.0
3,52cdef7c4bab8bd675297d8d,Digg,2004.0,news,$45M,NaN
4,52cdef7c4bab8bd675297d91,Geni,2006.0,web,$16.5M,NaN
5,52cdef7c4bab8bd675297d90,Postini,1999.0,web,$0,NaN
6,52cdef7c4bab8bd675297d92,Flektor,NaN,games_video,$0,NaN
7,52cdef7c4bab8bd675297d93,Fox Interactive Media,1979.0,web,$0,NaN
8,52cdef7c4bab8bd675297d96,Gizmoz,2003.0,web,$18.1M,NaN
9,52cdef7c4bab8bd675297d98,Slacker,2006.0,music,$73.1M,NaN


In [128]:
num_offices = []
for i in range(len(companies)):
    num_offices.append(len(companies[i]['offices']))
# print(len(num_offices)) # 14082
# max(num_offices) # 12
companies_df['num_offices'] = num_offices

In [130]:
for i in range(max(num_offices)):
    companies_df[f'office_{i+1}_longitude'] = [np.nan] * len(num_offices)
    companies_df[f'office_{i+1}_latitude'] = [np.nan] * len(num_offices)
    companies_df[f'office_{i+1}_location'] = [{}] * len(num_offices)

In [131]:
companies_data = []
for i in range(len(companies)):
    if len(companies[i]['offices'])==0:
        companies_data.append([[0, np.nan, np.nan]])
    elif len(companies[i]['offices'])>0:
        aux_offices = []
        for j in range(len(companies[i]['offices'])):
            aux_offices.append([j+1, companies[i]['offices'][j]['latitude'],
                                companies[i]['offices'][j]['longitude']])
        companies_data.append(aux_offices)
len(companies_data)

14082

In [132]:
for i in range(len(companies_data)):
    for j in range(len(companies_data[i])):
        if companies_data[i][j][0] != 0:
            companies_df.at[i, f'office_{j+1}_longitude'] = companies_data[i][j][2]
            companies_df.at[i, f'office_{j+1}_latitude'] = companies_data[i][j][1]

In [133]:
def getLocation(longitude,latitude):
    loc = {
        'type':'Point',
        'coordinates':[longitude, latitude]
    }
    return loc

In [135]:
for i in range(len(companies_data)):
    for j in range(len(companies_data[i])):
        if companies_data[i][j][0] != 0:
            companies_df.at[i, f'office_{j+1}_location'] = getLocation(companies_df.at[i, f'office_{j+1}_longitude'],
                                                                       companies_df.at[i, f'office_{j+1}_latitude'])

In [ ]:
# Nobody in the company likes to have companies with more than 10 years in a radius of 2 KM.
# Developers like to be near successful tech startups that have raised at least 1 Million dollars.
# Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
# 30% of the company have at least 1 child.
# Executives like Starbucks A LOT. Ensure there's a starbucks not to far.
# Account managers need to travel a lot
# All people in the company have between 25 and 40 years, give them some place to go to party.
# The CEO is Vegan

In [ ]:
# Nobody in the company likes to have companies with more than 10 years in a radius of 2 KM.




# {founded_year:{$gt: 2009}} # 146
# {founded_year:{$lte: 2009}} # 12990 (no quieres tenerlas cerca)
# Año en que quebraron

# https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude/43211266#43211266
# import geopy.distance
# coords_1 = (52.2296756, 21.0122287)
# coords_2 = (52.406374, 16.9251681)
# print geopy.distance.vincenty(coords_1, coords_2).km

# Al parecer hay tambien un operador near en mongodb (lo tenemos en los apuntes,
# poner primero el field de la manera que lo quier Mongo)

# Tabla limpia a mongo Db para hacer querys?

In [ ]:
# Developers like to be near successful tech startups that have raised at least 1 Million dollars.
# category_code
# founded_year
# total_money_raised (o dentro de las rondas: funding_rounds -> raised_amount & raised_currency_code)
# ipo.valuation_amount # no vale para startups -> Dinero por el que salen a bolsa
# Año en que quebraron, porque si quebraron no hay que tenerlas en cuenta